In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup as bs
import os
from splinter import Browser
import time
import re
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "Firefox\geckodriver.exe"}
    return Browser("firefox", **executable_path, headless=False)

# NASA Mars News
### Collect latest news title and paragraph

In [3]:
browser = init_browser()
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [4]:
results = soup.find_all("div", class_="list_text")

In [5]:
titles = []
paragraphs = []
for result in results:
    title = result.find('div', class_="content_title").text
    paragraph = result.find('div', class_="article_teaser_body").text
    titles.append(title)
    paragraphs.append(paragraph)

In [6]:
news_title = titles[0]
news_p = paragraphs[0]
news_title

"The Detective Aboard NASA's Perseverance Rover"

# Mars Featured Image
### Collect the featured image on JPL

In [15]:
browser = init_browser()
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

In [27]:
results = soup.find_all("article", class_="carousel_item")
results

[<article alt="Installing Hubble's New Camera" class="carousel_item" style="background-image: url('/spaceimages/images/wallpaper/PIA22911-1920x1200.jpg');">
 <div class="default floating_text_area ms-layer">
 <h2 class="category_title">
 </h2>
 <h2 class="brand_title">
 				  FEATURED IMAGE
 				</h2>
 <h1 class="media_feature_title">
 				  Installing Hubble's New Camera				</h1>
 <div class="description">
 </div>
 <footer>
 <a class="button fancybox" data-description="This image of NASAs Hubble Space Telescope shows Astronaut Jeffrey Hoffman and Story Musgrave installing the Wide Field and Planetary Camera 2 (WFPC2) on the Hubble Space Telescope, during SM1 in December, 1993." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA22911_ip.jpg" data-link="/spaceimages/details.php?id=PIA22911" data-title="Installing Hubble's New Camera" id="full_image">
 					FULL IMAGE
 				  </a>
 </footer>
 </div>
 <div class="gradient_container_top"></div>
 <div class="

In [35]:
new_result = results[0].find("a")
new_result

<a class="button fancybox" data-description="This image of NASAs Hubble Space Telescope shows Astronaut Jeffrey Hoffman and Story Musgrave installing the Wide Field and Planetary Camera 2 (WFPC2) on the Hubble Space Telescope, during SM1 in December, 1993." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA22911_ip.jpg" data-link="/spaceimages/details.php?id=PIA22911" data-title="Installing Hubble's New Camera" id="full_image">
					FULL IMAGE
				  </a>

In [39]:
img = new_result["data-fancybox-href"]

In [40]:
featured_img_url = "https://www.jpl.nasa.gov" + img
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA22911_ip.jpg'

# Mars Weather
### Scrape the weather information from Twitter

In [3]:
browser = init_browser()
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, "html.parser")

In [5]:
results = soup.find_all("article")
results

ci r-11cpok1 r-1ny4l3l r-bztko3 r-lrvibr" data-focusable="true" data-testid="caret" role="button" tabindex="0"><div class="css-901oao r-1awozwy r-1re7ezh r-6koalj r-1qd0xha r-a023e6 r-16dba41 r-1h0z5md r-ad9z0x r-bcqeeo r-o7ynqc r-clp7b1 r-3s2u2q r-qvutc0" dir="ltr"><div class="css-1dbjc4n r-xoduu5"><div class="css-1dbjc4n r-1niwhzg r-sdzlij r-1p0dtai r-xoduu5 r-1d2f490 r-podbf7 r-u8s1d r-zchlnj r-ipm5af r-o7ynqc r-6416eg"></div><svg class="r-4qtqp9 r-yyyyoo r-ip8ujx r-dnmrzs r-bnwqim r-1plcrui r-lrvibr r-27tl0q" viewbox="0 0 24 24"><g><path d="M20.207 8.147c-.39-.39-1.023-.39-1.414 0L12 14.94 5.207 8.147c-.39-.39-1.023-.39-1.414 0-.39.39-.39 1.023 0 1.414l7.5 7.5c.195.196.45.294.707.294s.512-.098.707-.293l7.5-7.5c.39-.39.39-1.022 0-1.413z"></path></g></svg></div></div></div></div></div></div></div><div class="css-1dbjc4n"><div class="css-1dbjc4n"><div class="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0" dir="auto" lang="en"><span class="css-901o

In [6]:
new_results = results[0].find_all("span", class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0")
mars_weather = new_results[4].text
mars_weather

'InSight sol 535 (2020-05-29) low -91.3ºC (-132.4ºF) high -2.7ºC (27.2ºF)\nwinds from the SW at 5.2 m/s (11.5 mph) gusting to 16.7 m/s (37.3 mph)\npressure at 7.20 hPa'

# Mars Facts
### Scrape table with pandas

In [86]:
# Using Pandas to pull an HTML table on Mars facts
browser = init_browser()
url = "https://space-facts.com/mars/"
browser.visit(url)

In [87]:
html = browser.html
soup = bs(html, "html.parser")

In [90]:
results = soup.find("table", id="tablepress-p-mars-no-2")
results

<table class="tablepress tablepress-id-p-mars" id="tablepress-p-mars-no-2"><tbody><tr class="row-1 odd"><td class="column-1"><strong>Equatorial Diameter:</strong></td><td class="column-2">6,792 km<br/></td></tr><tr class="row-2 even"><td class="column-1"><strong>Polar Diameter:</strong></td><td class="column-2">6,752 km<br/></td></tr><tr class="row-3 odd"><td class="column-1"><strong>Mass:</strong></td><td class="column-2">6.39 × 10^23 kg<br/> (0.11 Earths)</td></tr><tr class="row-4 even"><td class="column-1"><strong>Moons:</strong></td><td class="column-2">2 (<a href="https://space-facts.com/moons/phobos/">Phobos</a> &amp; <a href="https://space-facts.com/moons/deimos/">Deimos</a>)</td></tr><tr class="row-5 odd"><td class="column-1"><strong>Orbit Distance:</strong></td><td class="column-2">227,943,824 km<br/> (1.38 AU)</td></tr><tr class="row-6 even"><td class="column-1"><strong>Orbit Period:</strong></td><td class="column-2">687 days (1.9 years)<br/></td></tr><tr class="row-7 odd"><t

In [93]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [95]:
df = tables[0]
df.columns = ["Topic", "Fact"]
df.head()

,Topic,Fact
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [96]:
fact_table = df.to_html()
fact_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Topic</th>\n      <th>Fact</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>

# Mars Hemispheres
### Obtain images for each of Mars' hemispheres

In [104]:
browser = init_browser()
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [105]:
html = browser.html
soup = bs(html, "html.parser")

In [110]:
results = soup.find_all("div", class_="item")
results

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [122]:
hrefs = []
for result in results:
    href = result.find("a")
    hrefs.append(href["href"])
hrefs

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [135]:
re.split("/search/map/Mars/Viking/|_enhanced", hrefs[2])

['', 'syrtis_major', '']

In [137]:
url = "https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced"
browser = init_browser()
browser.visit(url)

In [139]:
html = browser.html
soup = bs(html, "html.parser")

In [141]:
results = soup.find("img", class_="wide-image")
results["src"]

'/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

In [143]:
url = "https://astrogeology.usgs.gov"
hemisphere_images_urls = []
for href in hrefs:
    browser = init_browser()
    browser.visit(url + href)
    time.sleep(1)
    html = browser.html
    soup = bs(html, "html.parser")

    res = re.split("/search/map/Mars/Viking/|_enhanced", href)
    title = res[1]

    results = soup.find("img", class_="wide-image")
    img = results["src"]
    img_url = url + img

    hemisphere_images_urls.append({"title" : title, "img_url" : img_url})

hemisphere_images_urls

[{'title': 'cerberus',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'schiaparelli',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'syrtis_major',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'valles_marineris',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]